## Prediction model

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [25]:
salary_df = pd.read_csv("dataset_for_prediction.csv")
salary_df[(salary_df["Year"]==2000) & (salary_df["Tm"]=="SEA")]

,Unnamed: 0,Year,Player,Pos,Age,Tm,G,MP,PER,TS%,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Salaries,Rank
5,5,2000,Brent Barry,SG,28.0,SEA,80.0,2726.0,14.6,0.609,...,322.0,372.0,291.0,103.0,31.0,142.0,228.0,945.0,3960000.0,58
34,34,2000,Greg Foster,PF,31.0,SEA,60.0,718.0,6.9,0.430,...,91.0,107.0,41.0,10.0,18.0,28.0,105.0,203.0,857000.0,135


In [4]:


data_dum = pd.get_dummies(columns=["Pos", "Tm"], data=salary_df)
prediction_data = data_dum.drop(columns = ["Player"])
print(prediction_data)

       Unnamed: 0  Year   Age     G      MP   PER    TS%   3PAr    FTr  ORB%  \
0               0  2000  24.0  82.0  3070.0  20.6  0.570  0.288  0.282   3.2   
1               1  2000  23.0  27.0   361.0   4.3  0.310  0.147  0.042   1.6   
2               2  2000  29.0  82.0  2593.0  17.4  0.524  0.223  0.257   2.3   
3               3  2000  31.0  82.0  2590.0  19.5  0.542  0.360  0.221   2.7   
4               4  2000  30.0  59.0  1173.0  10.2  0.481  0.011  0.389   6.2   
5               5  2000  28.0  80.0  2726.0  14.6  0.609  0.564  0.222   2.0   
6               6  2000  19.0  24.0   130.0   7.6  0.397  0.171  0.343   3.6   
7               7  2000  27.0  82.0  1691.0  16.5  0.569  0.166  0.339   1.1   
8               8  2000  21.0  82.0  3155.0  16.7  0.521  0.206  0.242   2.8   
9               9  2000  32.0  73.0  2459.0  15.1  0.468  0.360  0.114   2.2   
10             10  2000  22.0  46.0   423.0  10.0  0.463  0.009  0.330   5.7   
11             11  2000  20.0  81.0  299

In [5]:
from sklearn.model_selection import train_test_split
X = prediction_data.copy().drop(columns=["Salaries"])
y = prediction_data["Salaries"]

X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.33, random_state=42)

### Predicting using Random Forest ###

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

# TODO
rf = RandomForestClassifier(n_estimators=100, max_depth=4,random_state=0).fit(X_train, y_train)

In [7]:
rayAllen = data_dum[data_dum["Player"] == "Kobe Bryant"]
rayAllen = rayAllen.drop(columns=["Player", "Salaries"])
rayAllen_pred = rf.predict(rayAllen)

### Predicting using Linear Regression ###

In [10]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, y_train)
print(lr.coef_)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [29]:
print(lr.predict(rayAllen[rayAllen["Year"] == 2005]))
print(salary_df[salary_df["Player"] == "Kobe Bryant"]["Salaries"][salary_df["Year"] == 2005])

[12687058.78066832]
2114    14175000.0
Name: Salaries, dtype: float64


### Predicting Key Players Salaries ###
#### Predicting new contract salary by year ####
Chosen players:
* Anthony Davis
* Damian Lillard
* Giannis Antetokounmpo

In [48]:
players = ["Anthony Davis", "Damian Lillard", "Giannis Antetokounmpo"]
choosenModel = lr
for player in players:
    predictedSalary = choosenModel.predict(data_dum[(data_dum["Player"] == player) & (salary_df["Year"] == 2019 )].drop(columns=["Player", "Salaries"]))
    salaryNow = salary_df[(salary_df["Player"] == player) & (salary_df["Year"] == 2019 )]["Salaries"].iloc[0]
    print("Player: " + str(player)+"; "+ "Current Salary: "+ str(salaryNow) +"; "+"Predicted next season salary: "+ str(predictedSalary))
    



Player: Anthony Davis; Current Salary: 25434263.0; Predicted next season salary: [19987990.6008805]
Player: Damian Lillard; Current Salary: 27977689.0; Predicted next season salary: [19403331.80264282]
Player: Giannis Antetokounmpo; Current Salary: 24157304.0; Predicted next season salary: [22772073.40977889]


In [28]:
teams = salary_df[["Tm", "Year", "Salaries"]]
teams = teams.groupby(["Tm", "Year"]).sum()
teams.rename(columns={'Salaries':'Teamcap'}, inplace=True)
test = pd.merge(left=salary_df, right=teams, left_on=["Tm", "Year"], right_on=["Tm", "Year"])
test['Wage%'] = round(test['Salaries'] / test['Teamcap'] * 100, 2)

In [29]:
test

,Unnamed: 0,Year,Player,Pos,Age,Tm,G,MP,PER,TS%,...,AST,STL,BLK,TOV,PF,PTS,Salaries,Rank,Teamcap,Wage%
0,0,2000,Ray Allen,SG,24.0,MIL,82.0,3070.0,20.6,0.570,...,308.0,110.0,19.0,183.0,187.0,1809.0,9000000.0,14,28811000.0,31.24
1,1,2000,Rafer Alston,PG,23.0,MIL,27.0,361.0,4.3,0.310,...,70.0,12.0,0.0,29.0,29.0,60.0,301000.0,144,28811000.0,1.04
2,18,2000,Sam Cassell,PG,30.0,MIL,81.0,2899.0,21.1,0.551,...,729.0,102.0,8.0,267.0,255.0,1506.0,3500000.0,64,28811000.0,12.15
3,61,2000,Ervin Johnson,C,32.0,MIL,80.0,2129.0,12.7,0.550,...,44.0,81.0,127.0,80.0,298.0,383.0,2951000.0,76,28811000.0,10.24
4,108,2000,Glenn Robinson,SF,27.0,MIL,81.0,2909.0,17.8,0.534,...,193.0,78.0,41.0,223.0,212.0,1693.0,7250000.0,23,28811000.0,25.16
5,124,2000,Robert Traylor,PF,22.0,MIL,44.0,447.0,15.8,0.517,...,20.0,25.0,25.0,27.0,79.0,157.0,2059000.0,97,28811000.0,7.15
6,133,2000,Scott Williams,C,31.0,MIL,68.0,1488.0,15.6,0.539,...,28.0,40.0,66.0,65.0,230.0,520.0,3750000.0,62,28811000.0,13.02
7,2,2000,Kenny Anderson,PG,29.0,BOS,82.0,2593.0,17.4,0.524,...,420.0,139.0,8.0,130.0,230.0,1149.0,6680000.0,30,26829000.0,24.90
8,78,2000,Walter McCarty,PF,25.0,BOS,61.0,879.0,7.2,0.451,...,70.0,24.0,23.0,67.0,83.0,229.0,2500000.0,87,26829000.0,9.32
9,98,2000,Paul Pierce,SF,22.0,BOS,73.0,2583.0,19.8,0.550,...,221.0,152.0,62.0,178.0,237.0,1427.0,1503000.0,115,26829000.0,5.60
